In [ ]:
import os
import glob
import numpy as np
import xarray as xr
import pandas as pd
from sublimpy import utils, extrautils
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')

# Open velocity data and apply monthly planar fits

## SOS

In [ ]:
# Open all files, get variables we want, and concatenate into a single dataset
dir_sos = "/Users/elischwat/Development/data/sublimationofsnow/sosqc/sos_qc_geo_tiltcor_v20240307"
file_list_sos = sorted(glob.glob(os.path.join(dir_sos, "*.nc")))
vars_sos = [
        'dir_3m_c', 'SF_avg_1m_ue', 'SF_avg_2m_ue',
        'w_3m_ue',    'w_10m_ue', 'w_3m_d',     'w_10m_d', 'w_3m_uw',    'w_10m_uw',  
            'w_3m_c',     'w_5m_c',     'w_10m_c',    'w_15m_c',    'w_20m_c',
        'u_3m_ue',    'u_10m_ue', 'u_3m_d',     'u_10m_d', 'u_3m_uw',    'u_10m_uw',  
            'u_3m_c',     'u_5m_c',     'u_10m_c',    'u_15m_c',    'u_20m_c',
        'v_3m_ue',    'v_10m_ue', 'v_3m_d',     'v_10m_d', 'v_3m_uw',    'v_10m_uw',  
            'v_3m_c',     'v_5m_c',     'v_10m_c',    'v_15m_c',    'v_20m_c',
            
    ]

datasets = []
for file in file_list_sos:
    ds = xr.open_dataset(file)
    # this ensures we don't access variables that aren't in this dataset, which would throw an error
    ds_new = ds[set(ds.data_vars).intersection(vars_sos)]
    datasets.append(ds_new)
sos_ds = xr.concat(datasets, dim='time')
sos_ds = utils.fill_missing_timestamps(sos_ds)

In [ ]:
# Iterate over each month, and EC instruments, calculating and applying planar fits to u, v, w data, and combining the monthly datasets into a single dataset
ec_suffixes = [
    '3m_ue',    '10m_ue', '3m_d',     '10m_d', '3m_uw',    '10m_uw',  
    '3m_c',     '5m_c',     '10m_c',    '15m_c',    '20m_c'
]
planar_fitted_ds_ls = []
for month, year in [
    (11, 2022),
    (12, 2022),
    (1, 2023),
    (2, 2023),
    (3, 2023),
    (4, 2023),
    (5, 2023),
    (6, 2023),
]:
    this_ds = sos_ds.sel(time = sos_ds.time.dt.year.isin([year]))
    this_ds = this_ds.sel(time = this_ds.time.dt.month.isin([month]))
    for suffix in ec_suffixes:
        (a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(this_ds[f"u_{suffix}"], this_ds[f"v_{suffix}"], this_ds[f"w_{suffix}"])
        (u_streamwise, v_streamwise, w_streamwise) = extrautils.apply_planar_fit(
            this_ds[f"u_{suffix}"], this_ds[f"v_{suffix}"], this_ds[f"w_{suffix}"],
            a, W_f
        )
        print(f"{month}\t{year}\t{suffix}\t{round(np.rad2deg(tilt), 5)}")
        this_ds[f"u_{suffix}"] = ('time', u_streamwise)
        this_ds[f"v_{suffix}"] = ('time', v_streamwise)
        this_ds[f"w_{suffix}"] = ('time', w_streamwise)
    planar_fitted_ds_ls.append(this_ds)
sos_ds_fitted = xr.concat(planar_fitted_ds_ls, dim='time')

## SPLASH

In [ ]:
# Open all files, get variables we want, and concatenate into a single dataset for both AVP and KPS sites

# SPLASH KETTLE PONDS data
splash_kps_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/asfs/ASFS-30_Level2_SPLASH2021-2023/sledseb.asfs30.level2.0.10min.**.nc")
splash_kps_files = sorted([f for f in splash_kps_files 
    if '202211' in f 
    or '202212' in f 
    or '202301' in f 
    or '202302' in f 
    or '202303' in f 
    or '202304' in f 
    or '202305' in f 
    or '202306' in f 
])
datasets = [xr.open_dataset(file)[['wspd_u_mean', 'wspd_v_mean', 'wspd_w_mean']] for file in splash_kps_files]
splash_kps_ds = xr.concat(datasets, dim='time')
splash_kps_ds = utils.fill_missing_timestamps(splash_kps_ds)

# SPLASH AVERY PICNIC data
splash_avp_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/asfs/ASFS-50_Level2_SPLASH2021-2023/sledseb.asfs50.level2.0.10min.**.nc")
splash_avp_files = sorted([f for f in splash_avp_files 
    if '202211' in f 
    or '202212' in f 
    or '202301' in f 
    or '202302' in f 
    or '202303' in f 
    or '202304' in f 
    or '202305' in f 
    or '202306' in f 
])
datasets = [xr.open_dataset(file)[['wspd_u_mean', 'wspd_v_mean', 'wspd_w_mean']] for file in splash_avp_files]
splash_avp_ds = xr.concat(datasets, dim='time')
splash_avp_ds = utils.fill_missing_timestamps(splash_avp_ds)

In [ ]:
# KPS SITE Iterate over each month, calculating and applying planar fits to u, v, w data, and combining the monthly datasets into a single dataset
planar_fitted_ds_ls = []
for month, year in [
    (11, 2022),
    (12, 2022),
    (1, 2023),
    (2, 2023),
    (3, 2023),
    (4, 2023),
    (5, 2023),
    (6, 2023),
]:
    this_ds = splash_kps_ds.sel(time = splash_kps_ds.time.dt.year.isin([year]))
    this_ds = this_ds.sel(time = this_ds.time.dt.month.isin([month]))
    (a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(this_ds[f"wspd_u_mean"], this_ds[f"wspd_v_mean"], this_ds[f"wspd_w_mean"])
    (u_streamwise, v_streamwise, w_streamwise) = extrautils.apply_planar_fit(
        this_ds[f"wspd_u_mean"], this_ds[f"wspd_v_mean"], this_ds[f"wspd_w_mean"],
        a, W_f
    )
    this_ds[f"wspd_u_mean"] = ('time', u_streamwise)
    this_ds[f"wspd_v_mean"] = ('time', v_streamwise)
    this_ds[f"wspd_w_mean"] = ('time', w_streamwise)
    planar_fitted_ds_ls.append(this_ds)
splash_kps_ds_fitted = xr.concat(planar_fitted_ds_ls, dim='time')

In [ ]:
# AVP SITE Iterate over each month, calculating and applying planar fits to u, v, w data, and combining the monthly datasets into a single dataset
planar_fitted_ds_ls = []
for month, year in [
    (11, 2022),
    (12, 2022),
    (1, 2023),
    (2, 2023),
    (3, 2023),
    (4, 2023),
    (5, 2023),
    (6, 2023),
]:
    this_ds = splash_avp_ds.sel(time = splash_avp_ds.time.dt.year.isin([year]))
    this_ds = this_ds.sel(time = this_ds.time.dt.month.isin([month]))
    (a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(this_ds[f"wspd_u_mean"], this_ds[f"wspd_v_mean"], this_ds[f"wspd_w_mean"])
    (u_streamwise, v_streamwise, w_streamwise) = extrautils.apply_planar_fit(
        this_ds[f"wspd_u_mean"], this_ds[f"wspd_v_mean"], this_ds[f"wspd_w_mean"],
        a, W_f
    )
    this_ds[f"wspd_u_mean"] = ('time', u_streamwise)
    this_ds[f"wspd_v_mean"] = ('time', v_streamwise)
    this_ds[f"wspd_w_mean"] = ('time', w_streamwise)
    planar_fitted_ds_ls.append(this_ds)
splash_avp_ds_fitted = xr.concat(planar_fitted_ds_ls, dim='time')

## SAIL

In [ ]:
import act

In [ ]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")
ecor_gothic = 'guc30ecorM1.b1'
ecor_kp = 'guc30ecorS3.b1'
met_gothic = 'gucmetM1.b1'
start_date = '20221030'
end_date = '20230701'
output_dir = '/Users/elischwat/Development/data/sublimationofsnow/'

kp_sail_ecor_download_dir = os.path.join(output_dir, ecor_kp)
gothic_sail_ecor_download_dir = os.path.join(output_dir, ecor_gothic)

In [ ]:
act.discovery.download_arm_data(
    username,    token,    ecor_gothic,    
    start_date, end_date,
    output = gothic_sail_ecor_download_dir
)
act.discovery.download_arm_data(
    username,    token,    ecor_kp,    
    start_date, end_date,
    output = kp_sail_ecor_download_dir
)

In [ ]:
sail_gothic_ds = act.io.read_arm_netcdf(
    glob.glob(os.path.join(gothic_sail_ecor_download_dir, '*.cdf'))
)[['mean_u', 'mean_v', 'mean_w']]

sail_kps_ds = act.io.read_arm_netcdf(
    glob.glob(os.path.join(kp_sail_ecor_download_dir, '*.cdf'))
)[['mean_u', 'mean_v', 'mean_w']]

In [ ]:
# GOTHIC SITE Iterate over each month, calculating and applying planar fits to u, v, w data, and combining the monthly datasets into a single dataset
planar_fitted_ds_ls = []
for month, year in [
    (11, 2022),
    (12, 2022),
    (1, 2023),
    (2, 2023),
    (3, 2023),
    (4, 2023),
    (5, 2023),
    (6, 2023),
]:
    this_ds = sail_gothic_ds.sel(time = sail_gothic_ds.time.dt.year.isin([year]))
    this_ds = this_ds.sel(time = this_ds.time.dt.month.isin([month]))
    (a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(this_ds[f"mean_u"], this_ds[f"mean_v"], this_ds[f"mean_w"])
    (u_streamwise, v_streamwise, w_streamwise) = extrautils.apply_planar_fit(
        this_ds[f"mean_u"], this_ds[f"mean_v"], this_ds[f"mean_w"],
        a, W_f
    )
    this_ds[f"mean_u"] = ('time', u_streamwise)
    this_ds[f"mean_v"] = ('time', v_streamwise)
    this_ds[f"mean_w"] = ('time', w_streamwise)
    planar_fitted_ds_ls.append(this_ds)
sail_gothic_ds_fitted = xr.concat(planar_fitted_ds_ls, dim='time')

In [ ]:
# GOTHIC SITE Iterate over each month, calculating and applying planar fits to u, v, w data, and combining the monthly datasets into a single dataset
planar_fitted_ds_ls = []
for month, year in [
    (11, 2022),
    (12, 2022),
    (1, 2023),
    (2, 2023),
    (3, 2023),
    (4, 2023),
    (5, 2023),
    (6, 2023),
]:
    this_ds = sail_kps_ds.sel(time = sail_kps_ds.time.dt.year.isin([year]))
    this_ds = this_ds.sel(time = this_ds.time.dt.month.isin([month]))
    (a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(this_ds[f"mean_u"], this_ds[f"mean_v"], this_ds[f"mean_w"])
    (u_streamwise, v_streamwise, w_streamwise) = extrautils.apply_planar_fit(
        this_ds[f"mean_u"], this_ds[f"mean_v"], this_ds[f"mean_w"],
        a, W_f
    )
    this_ds[f"mean_u"] = ('time', u_streamwise)
    this_ds[f"mean_v"] = ('time', v_streamwise)
    this_ds[f"mean_w"] = ('time', w_streamwise)
    planar_fitted_ds_ls.append(this_ds)
sail_kps_ds_fitted = xr.concat(planar_fitted_ds_ls, dim='time')

# Conform all data to similar format

In [575]:
sos_df = sos_ds_fitted.to_dataframe()
sos_df_local = utils.modify_df_timezone(sos_df.reset_index(), 'UTC', 'US/Mountain')
sos_df_local = sos_df_local.set_index('time').resample('30min').mean().reset_index()

In [576]:
splash_avp_df = splash_avp_ds_fitted.to_dataframe()
splash_avp_df_local = utils.modify_df_timezone(splash_avp_df.reset_index(), 'UTC', 'US/Mountain')
splash_avp_df_local = splash_avp_df_local.set_index('time').resample('30min').mean().reset_index()

splash_kps_df = splash_kps_ds_fitted.to_dataframe()
splash_kps_df_local = utils.modify_df_timezone(splash_kps_df.reset_index(), 'UTC', 'US/Mountain')
splash_kps_df_local = splash_kps_df_local.set_index('time').resample('30min').mean().reset_index()

In [577]:
sail_gothic_df = sail_gothic_ds_fitted.to_dataframe()
sail_gothic_df_local = utils.modify_df_timezone(sail_gothic_df.reset_index(), 'UTC', 'US/Mountain')

sail_kps_df = sail_kps_ds_fitted.to_dataframe()
sail_kps_df_local = utils.modify_df_timezone(sail_kps_df.reset_index(), 'UTC', 'US/Mountain')

In [595]:
w_combo_df = sos_df_local[['time', 'w_3m_c', 'w_20m_c']].rename(
    columns={
        'w_3m_c': 'w 3m KPS SOS',
        'w_20m_c': 'w 20m KPS SOS',
    }
).merge(
    splash_avp_df_local[['time', 'wspd_w_mean']].rename(columns={'wspd_w_mean': 'w AVP SPLASH'}),
    on='time'
).merge(
    splash_kps_df_local[['time', 'wspd_w_mean']].rename(columns={'wspd_w_mean': 'w KPS SPLASH'}),
    on='time'
).merge(
    sail_gothic_df_local[['time', 'mean_w']].rename(columns={'mean_w': 'w GOT SAIL'}),
    on='time'
).merge(
    sail_kps_df_local[['time', 'mean_w']].rename(columns={'mean_w': 'w KPS SAIL'}),
    on='time'
)

# PLOT: Vertical velocities

In [623]:
rule = alt.Chart().transform_calculate(y='0').mark_rule(strokeWidth=2).encode(y='y:Q')

## Seasonal analysis - upvalley vs downvalley winds

In [618]:
upvalley_wind_times = sos_df_local.query("dir_3m_c < 152").query("dir_3m_c > 92").time.values
downvalley_wind_times = sos_df_local.query("dir_3m_c < 342").query("dir_3m_c > 292").time.values

is_no_bs = (sos_df_local.set_index('time')[['SF_avg_1m_ue', 'SF_avg_2m_ue']] > 0).sum(axis=1)
no_bs_times = is_no_bs[is_no_bs == 0].index.values

In [637]:
rule + alt.Chart(
    w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][
        w_combo_df.time.isin(set(upvalley_wind_times).intersection(set(no_bs_times)))
    ]
).transform_fold(
    w_combo_df.columns.drop('time').drop('w KPS SPLASH'),
    as_=('height', 'value')
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('mean(value):Q'),
    alt.Color('height:N')
) |\
rule + alt.Chart(
    w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][
        w_combo_df.time.isin(set(downvalley_wind_times).intersection(set(no_bs_times)))
    ]
).transform_fold(
    w_combo_df.columns.drop('time').drop('w KPS SPLASH'),
    as_=('height', 'value')
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('mean(value):Q'),
    alt.Color('height:N')
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/1713523193.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/1713523193.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/1713523193.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/1713523193.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > '20221130'][w_combo_df.time < '20230505'][


alt.HConcatChart(...)

## Case studies

Make sure we can replicate the plots from Chapter 1

In [626]:
rule + alt.Chart(
    sos_df_local[sos_df_local.time > DATE_START][sos_df_local.time < DATE_END].set_index('time')[['w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c']].resample('30min').mean().reset_index()
).mark_line().transform_fold(
    ['w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c'],
    as_=['height', 'value']
).encode(
    alt.X('time:T'). axis().title(None),
    alt.Y('value:Q').title(['Vertical wind speed (m/s)']),
    alt.Color('height:O').scale(scheme='turbo', domain=[
        'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c'
    ])
).properties(width=300, height = 125)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/2935914290.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sos_df_local[sos_df_local.time > DATE_START][sos_df_local.time < DATE_END].set_index('time')[['w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c']].resample('30min').mean().reset_index()


alt.LayerChart(...)

In [640]:
DATE_START = '20230505'
DATE_END = '20230506'
rule + alt.Chart(
    w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]
).transform_fold(
    w_combo_df.columns.drop('time').drop('w KPS SPLASH'),
    as_=('height', 'value')
).transform_window(
    rolling_avg = 'mean(value)',
    frame=[-2,2],
    groupby = ['height']
).mark_line(color='#beddf4', opacity=0.75).encode(
    alt.X('time:T'),
    alt.Y('rolling_avg:Q'),
    alt.Color('height:N')
).properties(height = 300, width=500)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/594980858.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]


alt.LayerChart(...)

In [635]:
DATE_START = '20230415'
DATE_END = '20230416'
rule + alt.Chart(
    w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]
).transform_fold(
    w_combo_df.columns.drop('time').drop('w KPS SPLASH'),
    as_=('height', 'value')
).transform_window(
    rolling_avg = 'mean(value)',
    frame=[-2,2],
    groupby = ['height']
).mark_line(color='#beddf4', opacity=0.75).encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:N')
).properties(height = 200)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/3354033102.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]


alt.LayerChart(...)

In [636]:
DATE_START = '20230417'
DATE_END = '20230418'
rule + alt.Chart(
    w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]
).transform_fold(
    w_combo_df.columns.drop('time').drop('w KPS SPLASH'),
    as_=('height', 'value')
).transform_window(
    rolling_avg = 'mean(value)',
    frame=[-2,2],
    groupby = ['height']
).mark_line(color='#beddf4', opacity=0.75).encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:N')
).properties(height = 200)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_3208/3796060813.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  w_combo_df[w_combo_df.time > DATE_START][w_combo_df.time < DATE_END]


alt.LayerChart(...)